# Compilador R Markdown para Notebooks de Jupyter

Creado el **29 de febrero del 2024**
### *Luis Muñiz Valledcor*

In [ ]:
import os
from google.colab import drive
from ipywidgets import Checkbox, HTML, VBox, HBox, Button
from IPython.display import display
import time

#@markdown ## Preparación del compilador
#@markdown Al ejecutar esta celda, se instarán todos los paquetes, dependencias y definiciones necesarias, para compilar notebooks jupyter en **Google Colaboratory** mediante el renderizador que emplea **R** para construir documentos **R Markdown**.

#@markdown Los paquetes básicos de **R** necesarios para procesar documentos **R Markdwon** se instalan por defecto. Adicionalmente, puede especificar otros paquetes de **R** empleados en su documento.

#@markdown Especifique los nombres de los paquetes requeridos separados por coma.

dir_proyecto = "/content/drive/Shareddrives/Regresión/Tarea-2" #@param {type:"string"}
paquetes = "kableExtra, gridExtra, binom, FactoMineR, ca, gplots, xtable" #@param {type:"string"}

print("\n==================================================")
print("CONECTAR CON G-DRIVE")
print("===================================================\n")

drive.mount('/content/drive')

print("\n==================================================")
print("DEFINIR FUNCIONES DE LA CAPA DE COMPATIBILIDAD")
print("===================================================\n")

def instalar_paquetes_r(dir_lib, lista_paquetes):
    """
    Instala los paquetes de R en un directorio específico si no están ya instalados.

    :param dir_lib: La ruta del directorio donde se deben instalar los paquetes de R.
    :param lista_paquetes: Una lista de nombres de paquetes de R para instalar.
    """
    for paquete in lista_paquetes:
        paquete_instalado = False
        for item in os.listdir(dir_lib):
            if os.path.isdir(os.path.join(dir_lib, item)) and item == paquete:
                paquete_instalado = True
                break
        if not paquete_instalado:
            !R -e "install.packages('{paquete}', lib='{dir_lib}')"

    os.environ["R_LIBS_USER"] = dir_lib

def generar_yaml(paquetes_latex, latex_engine='xelatex', wrap='72', es_personalizado=False):
    """
    Genera el YAML para un documento R Markdown.

    Esta función permite especificar paquetes LaTeX que se incluirán en el preámbulo del documento y configura el motor de LaTeX y la opción de ajuste de texto (wrap) en el editor.

    Parámetros:
    - paquetes_latex (str): Una cadena de nombres de paquetes LaTeX separados por comas.
    - latex_engine (str, opcional): El motor de LaTeX a utilizar. Valor predeterminado: 'xelatex'.
    - wrap (str, opcional): El número de columnas para el ajuste de texto en el editor. Valor predeterminado: '72'.

    Retorna:
    - str: El YAML configurado como una cadena de texto.
    """
    # Divide la cadena de paquetes en una lista
    lista_paquetes = paquetes_latex.split(',')

    # Inicializa la cadena de header-includes
    header_includes = ""
    # Agrega cada paquete a la cadena de header-includes
    for paquete in lista_paquetes:
        header_includes += f"- \\usepackage{{{paquete.strip()}}}\n"
    # Añade el path para las imágenes
    if es_personalizado:
        header_includes += f"- \\graphicspath{{"+"{../../img/}"+"}\n" #Las compilaciones personalizadas están en un nivel más abajo
    else:
        header_includes += f"- \\graphicspath{{"+"{../img/}"+"}\n"
    # Construye el YAML completo
    yaml_front_matter = f"""
header-includes:
{header_includes}
output:
  pdf_document:
    latex_engine: {latex_engine}
    fig_caption: true
editor_options:
  markdown:
    wrap: {wrap}
"""
    return yaml_front_matter

def reemplazar_yaml(rmd_contenido, nuevo_yaml):
    """
    Esta función reemplaza el bloque YAML en un documento R Markdown sin eliminar
    las líneas delimitadoras '---'.

    :param rmd_contenido: El contenido del documento R Markdown.
    :param nuevo_yaml: El nuevo bloque YAML que se desea establecer.
    :return: El contenido actualizado del documento R Markdown.
    """
    # Divide el contenido en líneas
    lineas = rmd_contenido.split('\n')

    # Busca las líneas que contienen solo '---' que delimitan el YAML
    indices_delimitadores = [i for i, linea in enumerate(lineas) if linea.strip() == '---']

    # Verifica que se encontraron al menos dos delimitadores para un bloque YAML válido
    if len(indices_delimitadores) >= 2:
        # Reemplaza el contenido entre los delimitadores con el nuevo YAML
        lineas[indices_delimitadores[0] + 1:indices_delimitadores[1]] = nuevo_yaml.strip().split('\n')

    # Reconstruye y devuelve el contenido actualizado
    return '\n'.join(lineas)

def eliminar_yaml(rmd_contenido):
    """
    Esta función elimina el bloque YAML de un documento R Markdown, incluyendo
    las líneas delimitadoras '---'.

    :param rmd_contenido: El contenido del documento R Markdown.
    :return: El contenido actualizado del documento R Markdown sin el bloque YAML.
    """
    # Divide el contenido en líneas
    lineas = rmd_contenido.split('\n')

    # Busca las líneas que contienen solo '---' que delimitan el YAML
    indices_delimitadores = [i for i, linea in enumerate(lineas) if linea.strip() == '---']

    # Verifica que se encontraron al menos dos delimitadores para un bloque YAML válido
    if len(indices_delimitadores) >= 2:
        # Elimina el contenido entre los delimitadores, incluyendo los delimitadores mismos
        del lineas[indices_delimitadores[0]:indices_delimitadores[1] + 1]

    # Reconstruye y devuelve el contenido actualizado sin el bloque YAML
    return '\n'.join(lineas)

def preparar_rmd(ruta_rmd_lectura, ruta_rmd_escritura, paquetes_latex, latex_engine, wrap, eliminar=False, es_personalizado=False):
    """
    Esta función lee un archivo R Markdown, y dependiendo del valor del parámetro 'eliminar',
    modifica su bloque YAML o lo elimina por completo. Luego, guarda el contenido resultante en un nuevo archivo.

    :param ruta_rmd_lectura: El path del archivo R Markdown original para leer.
    :param ruta_rmd_escritura: El path del archivo R Markdown donde se guardará el contenido modificado.
    :param paquetes_latex: Lista de paquetes LaTeX que deben incluirse en el YAML. Solo necesario si eliminar=False.
    :param latex_engine: Motor de LaTeX a utilizar. Solo necesario si eliminar=False.
    :param wrap: Configuración de ajuste para el código en el YAML. Solo necesario si eliminar=False.
    :param eliminar: Booleano que indica si se debe eliminar el YAML (True) o reemplazarlo (False).
    """

    # Leer el contenido del archivo R Markdown original
    with open(ruta_rmd_lectura, 'r') as archivo:
        contenido_rmd = archivo.read()

    # Decidir entre reemplazar o eliminar el bloque YAML
    if eliminar:
        contenido_rmd = eliminar_yaml(contenido_rmd)
    else:
        nuevo_yaml = generar_yaml(paquetes_latex, latex_engine, wrap, es_personalizado)
        contenido_rmd = reemplazar_yaml(contenido_rmd, nuevo_yaml)

    # Ajustar los chunks de Python a R
    rmd_limpio = ajustar_chunks_python_a_r(contenido_rmd)

    # Escribir el contenido modificado en el nuevo archivo R Markdown
    with open(ruta_rmd_escritura, 'w') as archivo_nuevo:
        archivo_nuevo.write(rmd_limpio)

#Reemplazo, está en fase de prueba
def ajustar_chunks_python_a_r(contenido):
    """
    Ajusta los chunks de Python en un documento R Markdown para que sean compatibles con R,
    identificando y reemplazando los tokens en la primera línea del chunk.

    Parámetros:
    - contenido: El contenido del documento R Markdown.

    Retorna:
    - El contenido del documento con los chunks ajustados.
    """
    # Diccionario de tokens reconocibles y su correspondiente en R Markdown
    tokens = {
        "setup": "setup",
        "echo=TRUE": "echo=TRUE",
        "echo=T": "echo=T",
        "echo=FALSE": "echo=FALSE",
        "echo=F": "echo=F",
        "include=TRUE": "include=TRUE",
        "include=T": "include=T",
        "include=FALSE": "include=FALSE",
        "include=F": "include=F",
        "eval=TRUE": "eval=TRUE",
        "eval=T": "eval=T",
        "eval=FALSE": "eval=FALSE",
        "eval=F": "eval=F",
        "warning=FALSE": "warning=FALSE",
        "warning=F": "warning=F",
        "results='asis'": "results='asis'"
    }

    # Procesa cada línea del contenido para ajustar los chunks
    lineas = contenido.split('\n')
    lineas_ajustadas = []
    i = 0

    while i < len(lineas):
        linea = lineas[i].strip()

        # Omitir líneas que comienzan con los patrones de región específicos
        if linea.startswith("<!-- #region") or linea.startswith("<!-- #endregion"):
            # Simplemente incrementar el índice para no incluir esta línea en el resultado
            i += 1
            continue

        # Encuentra el inicio de un chunk de Python
        if linea.startswith("```{python"):
            i += 1  # Avanza a la siguiente línea para leer los tokens
            if i < len(lineas):
                primera_linea = lineas[i].replace(" ", "")
                opciones_chunk = []
                # Busca y agrega los tokens encontrados en las opciones del chunk de R
                for token in tokens:
                    if token in primera_linea:
                        opciones_chunk.append(tokens[token])
                        primera_linea = primera_linea.replace(token, "", 1)

                # Reescribe el chunk como uno de R con las opciones encontradas
                if opciones_chunk:
                    lineas_ajustadas.append(f"```{{r, {', '.join(opciones_chunk)}}}")
                else:
                    # Si no encuentra tokens, mantiene el chunk como está
                    lineas_ajustadas.append("```{r}")
                    i -= 1  # Vuelve a revisar la misma línea en la siguiente iteración si no se encontraron tokens.
        else:
            # Añade la línea sin cambios si no es el inicio de un chunk
            lineas_ajustadas.append(linea)
        i += 1

    return '\n'.join(lineas_ajustadas)

print("\n==================================================")
print("INSTALAR HERRAMIENTAS Y DEPENDENCIAS EN EL SISTEMA")
print("===================================================\n")

!apt-get update
#!apt-get install r-base
!apt-get install texlive-xetex texlive-fonts-recommended texlive-latex-extra
!apt-get install pandoc
!pip install jupyter nbconvert
!pip install jupytext

print("\n==================================================")
print("INSTALAR PAQUETES DE R")
print("===================================================\n")

lista_paquetes = [paquete.strip() for paquete in paquetes.split(',')]
dir_lib = dir_proyecto + "/lib"

instalar_paquetes_r(dir_lib, lista_paquetes)

print("\n==================================================")
print("EL COMPILADOR ESTA LISTO")
print("====================================================\n")

In [ ]:
#@markdown ## Compilar Notebook final
#@markdown En esta celda se compilan todos los Notebooks del proyecto. El documento resultante se crea en el subdirectorio `build` del proyecto con el nombre especificado en el campo `nombre_documento`.
#@markdown  Para que la compilación sea efectiva, se requiere que todos los notebooks estén depurados y sólo se recomienda realizar la compilación cuando el documento sea estable en su totalidad.
#@markdown  Para etapas preliminares (cuando hay varios miembros del equipo editando simultáneamente sus notebooks de trabajo) se recomienda una compilación personalizada.
#@markdown Eso permite crear compilaciones sólo con los notebooks que interesan a cada miembro del equipo.

#@markdown ### Ruta del directorio de trabajo

#@markdown ### Nombre del documento final (nombre individual)
nombre_documento = "Tarea2" #@param {type:"string"}

#@markdown Tiempo de espera antes de iniciar la compilación.
retardo = 0 #@param {type:"integer"}

#@markdown ### Configuración YAML del documento R Markdown
#@markdown Incluye las configuraciones para el encabezado y opciones del documento.
latex_engine = "xelatex" #@param ["pdflatex", "xelatex", "lualatex"]
wrap = 72 #@param {type:"integer"}
paquetes_latex = "mathtools, amsmath" #@param {type:"string"}

dir_notebooks = dir_proyecto + "/notebooks"
dir_build = dir_proyecto + "/build/"
dir_build_jupytext = dir_proyecto + "/build/partial-build/original-jupytext"
dir_prep_build_rmarkdown = dir_proyecto + "/build/partial-build/prep-build-rmarkdown"

ruta_documento_Rmd_final = dir_build + nombre_documento + ".Rmd"

print("\n==================================================")
print("PREPARACIÓN DE LOS DIRECTORIOS PARA LA COMPILACIÓN")
print("====================================================\n")

#!rm -r "{dir_build}"
!mkdir -p "{dir_build}"
!mkdir -p "{dir_build_jupytext}"
!mkdir -p "{dir_prep_build_rmarkdown}"

time.sleep(retardo)

print("\n==================================================")
print("OBTENCIÓN DE TODOS LOS NOTEBOOKS DISPONIBLES")
print("====================================================\n")

# Comando para eliminar archivos específicos en dir_build
!find {dir_build} -name "{nombre_documento}.*" -delete

# Comando para eliminar todos los archivos en dir_build_jupytext
!rm -rf {dir_build_jupytext}/*

# Comando para eliminar todos los archivos en dir_prep_build_rmarkdown
!rm -rf {dir_prep_build_rmarkdown}/*

conservar_yalm=True
# Lista para almacenar las rutas de los archivos .ipynb
notebooks = []
# Lista para almacenar las rutas de los archivos .Rmd procesados individualmente
rmarkdowns = []

# Recopilar todos los archivos .ipynb
for root, dirs, files in os.walk(dir_notebooks):
    for file in sorted(files):  # Esto ordena los archivos en el nivel actual del directorio.
        if file.endswith(".ipynb"):
            ipynb_path = os.path.join(root, file)
            notebooks.append((ipynb_path, file))

# Ordenar la lista completa de notebooks en orden lexicográfico
notebooks.sort(key=lambda x: x[1])

print("\n==================================================")
print("CONVERSIÓN Y PREPARACIÓN PARA LA COMPILACIÓN")
print("====================================================\n")

# Procesar cada archivo .ipynb ordenadamente
for ipynb_path, file in notebooks:
    original_rmd_path = os.path.join(dir_build_jupytext, file.replace(".ipynb", ".Rmd"))
    prep_bild_rmd_path = os.path.join(dir_prep_build_rmarkdown, os.path.basename(original_rmd_path))

    # Aquí se deberías reemplazar el comando mágico ! por una llamada subprocess para mejor práctica
    # y asegurar que el script sea ejecutable en un contexto más amplio que solo Jupyter. Esto para más adelante
    !jupytext --to rmarkdown --output "$original_rmd_path" "$ipynb_path"

    if conservar_yalm:
        # La primera vez se modifica el YAML
        preparar_rmd(original_rmd_path, prep_bild_rmd_path, paquetes_latex, latex_engine, wrap, eliminar=False)
        conservar_yalm = False
    else:
        # Del segundo archivo en adelante se eliminan los YAML.
        preparar_rmd(original_rmd_path, prep_bild_rmd_path, paquetes_latex, latex_engine, wrap, eliminar=True)

    rmarkdowns.append(prep_bild_rmd_path)

# Abrir el archivo de salida en modo de escritura
with open(ruta_documento_Rmd_final, 'w') as salida:
    for ruta in rmarkdowns:
        # Asegurarse de que el archivo existe
        try:
            # Abrir el archivo actual en modo de lectura
            with open(ruta, 'r') as entrada:
                # Leer el contenido del archivo y escribirlo en el archivo de salida
                salida.write(entrada.read())
                # Aadir un salto de línea entre contenidos de archivos diferentes
                salida.write('\n')
        except FileNotFoundError:
            print(f"El archivo {ruta} no existe y será omitido.")

print("\n==================================================")
print("COMPILAR R MARKDOWN")
print("====================================================\n")

!R -e "rmarkdown::render('{ruta_documento_Rmd_final}', output_format='pdf_document')"


In [ ]:
#@markdown ## Seleccionar Notebooks para una compilación personalizada
#@markdown En esta celda se seleccionan los Notebooks para construir una compilación parcial del documento.
#@markdown La compilación resultante se almacena en un subdirectorio del mismo nombre especificado en el campo `nombre_documento`.
#@markdown La finalidad de esta sección es crear compilaciones personalizadas e independientes para cada integrante del equipo.
#@markdown Uno de los usos más útiles es la depuración de los notebooks de cada integrante, sin la intromisión de errores introducidos durante la edición de los notebooks por otros integrantes del equipo.

#@markdown ### Nombre del documento personalizado
nombre_documento = "compilación-rik" #@param {type:"string"}

#@markdown Tiempo de espera antes de iniciar la búsqueda.
retardo = 1 #@param {type:"integer"}

#@markdown ### Configuración YAML del documento R Markdown
#@markdown Incluye las configuraciones para el encabezado y opciones del documento.
latex_engine = "xelatex" #@param ["pdflatex", "xelatex", "lualatex"]
wrap = 72 #@param {type:"integer"}
paquetes_latex = "mathtools, amsmath" #@param {type:"string"}

dir_notebooks = dir_proyecto + "/notebooks"
dir_build = dir_proyecto + "/custom-build/" + nombre_documento + "/"
dir_build_jupytext = dir_build + "partial-build/original-jupytext"
dir_prep_build_rmarkdown = dir_build + "partial-build/prep-build-rmarkdown"

ruta_documento_Rmd_final = dir_build + nombre_documento + ".Rmd"

seleccionado_notebook=True

time.sleep(retardo)

# Lista para almacenar las rutas de los archivos .ipynb
notebooks = []

# Crear una lista para guardar los widgets checkbox
checkboxes = []

# Listar y ordenar alfabéticamente los archivos, luego crear un checkbox para cada archivo .ipynb
#archivos_ipynb = sorted([archivo for archivo in os.listdir(dir_notebooks) if archivo.endswith(".ipynb")])

# Recopilar todos los archivos .ipynb
for root, dirs, files in os.walk(dir_notebooks):
    for file in sorted(files):  # Esto ordena los archivos en el nivel actual del directorio.
        if file.endswith(".ipynb"):
            ipynb_path = os.path.join(root, file)
            notebooks.append((ipynb_path, file))

# Ordenar la lista completa de notebooks en orden lexicográfico
notebooks.sort(key=lambda x: x[1])

boton_todos = Button(description='Todos')
boton_ninguno = Button(description='Ninguno')

# Función para seleccionar todos los checkboxes
def seleccionar_todos(b):
    for checkbox in checkboxes:
        checkbox.value = True

# Función para deseleccionar todos los checkboxes
def deseleccionar_todos(b):
    for checkbox in checkboxes:
        checkbox.value = False

# Conexión de botones con sus funciones correspondientes
boton_todos.on_click(seleccionar_todos)
boton_ninguno.on_click(deseleccionar_todos)

# Creación de checkboxes para cada archivo .ipynb
seleccionado_notebook = True
for path, archivo in notebooks:
    checkbox = Checkbox(value=seleccionado_notebook, description=archivo)
    checkboxes.append(checkbox)
    seleccionado_notebook = False  # Después del primero, todos están deseleccionados

# Muestra los controles y las casillas de verificación centradas
controles = HBox([boton_todos, boton_ninguno], layout={'justify_content': 'center'})
lista_checkboxes = VBox(checkboxes, layout={'align_items': 'center'})
display(HTML('<div style="text-align: center;"><h3>Seleccione los notebooks para la construcción del documento:</h3></div>'))
display(VBox([controles, lista_checkboxes]))

In [ ]:
#@markdown ### Compilar Notebook personalizado
#@markdown Tiempo de espera antes de iniciar la compilación personalizada.

retardo = 10 #@param {type:"integer"}

print("\n==================================================")
print("PREPARACIÓN DE LOS DIRECTORIOS PARA LA COMPILACIÓN")
print("====================================================\n")

#!rm -r "{dir_build}"
!mkdir -p "{dir_build}"
!mkdir -p "{dir_build_jupytext}"
!mkdir -p "{dir_prep_build_rmarkdown}"

# Comando para eliminar archivos específicos en dir_build
!find {dir_build} -name "{nombre_documento}.*" -delete

# Comando para eliminar todos los archivos en dir_build_jupytext
!rm -rf {dir_build_jupytext}/*

# Comando para eliminar todos los archivos en dir_prep_build_rmarkdown
!rm -rf {dir_prep_build_rmarkdown}/*

time.sleep(retardo)

print("\n==================================================")
print("OBTENCIÓN DE TODOS LOS NOTEBOOKS SELECCIONADOS")
print("====================================================\n")

conservar_yalm=True

# Lista para almacenar las rutas de los archivos .Rmd procesados individualmente
rmarkdowns = []

# Iterar sobre los checkboxes y mostrar los nombres de los archivos seleccionados
archivos_seleccionados = [cb.description for cb in checkboxes if cb.value]

notebooks_seleccionados = [tupla for tupla in notebooks if tupla[1] in archivos_seleccionados]

print("\n==================================================")
print("CONVERSIÓN Y PREPARACIÓN PARA LA COMPILACIÓN")
print("====================================================\n")

# Procesar cada archivo .ipynb ordenadamente
for ipynb_path, file in notebooks_seleccionados:
    original_rmd_path = os.path.join(dir_build_jupytext, file.replace(".ipynb", ".Rmd"))
    prep_bild_rmd_path = os.path.join(dir_prep_build_rmarkdown, os.path.basename(original_rmd_path))

    # Aquí se deberías reemplazar el comando mágico ! por una llamada subprocess para mejor práctica
    # y asegurar que el script sea ejecutable en un contexto más amplio que solo Jupyter. Esto para más adelante
    !jupytext --to rmarkdown --output "$original_rmd_path" "$ipynb_path"

    if conservar_yalm:
        # La primera vez se modifica el YAML
        preparar_rmd(original_rmd_path, prep_bild_rmd_path, paquetes_latex, latex_engine, wrap, eliminar=False, es_personalizado=True)
        conservar_yalm = False
    else:
        # Del segundo archivo en adelante se eliminan los YAML.
        preparar_rmd(original_rmd_path, prep_bild_rmd_path, paquetes_latex, latex_engine, wrap, eliminar=True)

    rmarkdowns.append(prep_bild_rmd_path)

# Abrir el archivo de salida en modo de escritura
with open(ruta_documento_Rmd_final, 'w') as salida:
    # Iterar sobre la lista de archivos de entrada
    for ruta in rmarkdowns:
        # Asegurarse de que el archivo existe
        try:
            # Abrir el archivo actual en modo de lectura
            with open(ruta, 'r') as entrada:
                # Leer el contenido del archivo y escribirlo en el archivo de salida
                salida.write(entrada.read())
                # Añadir un salto de línea entre contenidos de archivos diferentes
                salida.write('\n')
        except FileNotFoundError:
            print(f"El archivo {ruta} no existe y será omitido.")

print("\n==================================================")
print("COMPILAR R MARKDOWN")
print("====================================================\n")

!R -e "rmarkdown::render('{ruta_documento_Rmd_final}', output_format='pdf_document')"
